<a href="https://colab.research.google.com/github/markiroberts/gogglecolab01/blob/main/03_pt02_Use_matches_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import pandas as pd
import numpy as np
from google.colab import drive
from google.colab import data_table
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

data_table.enable_dataframe_formatter()

directory_path = "/content/drive/MyDrive/ColabData/"
try:
  os.stat(directory_path)
except:
  print(f"Cant access {directory_path}")
  drive.mount('/content/drive')


filename = "/content/drive/MyDrive/ColabData/matches.csv"

match_df = pd.read_csv(filename, index_col=0)

match_df = match_df.sort_values(["season", "team", "date"])
match_df = match_df.reset_index(drop=True)

print(match_df[["season", "team", "opponent","date"]].head())
print(match_df[["season", "team", "opponent","date"]].tail())

print(f"Loaded {match_df.shape[0]} rows and {match_df.shape[1]} columns from {filename}")
print(f"Seasons : {sorted(match_df.season.unique())}")

   season         team        opponent        date
0    2020      Arsenal       Tottenham  2020-12-06
1    2020      Arsenal         Burnley  2020-12-13
2    2020      Arsenal  Crystal Palace  2021-05-19
3    2020      Arsenal        Brighton  2021-05-23
4    2020  Aston Villa       Tottenham  2021-05-19
      season                     team         opponent        date
1955    2023  Wolverhampton Wanderers  Nott'ham Forest  2023-12-09
1956    2023  Wolverhampton Wanderers         West Ham  2023-12-17
1957    2023  Wolverhampton Wanderers          Chelsea  2023-12-24
1958    2023  Wolverhampton Wanderers        Brentford  2023-12-27
1959    2023  Wolverhampton Wanderers          Everton  2023-12-30
Loaded 1960 rows and 25 columns from /content/drive/MyDrive/ColabData/matches.csv
Seasons : [2020, 2021, 2022, 2023]


In [71]:
def rolling_averages(group, cols, newcols, number):
  group = group.sort_values("dateandtime")
  rolling_stats = group[cols].rolling(number, closed='left').mean()
  try:
    group.drop(newcols, axis=1)
  except:
    a = 1

  group[newcols] = rolling_stats
  group = group.dropna(subset=newcols)
  return group

def rolling_average_columns(cols, number):
  new_cols = []
  for x in cols:
    y = f"{x}_rolling number_{number}"
    new_cols.append(y)
  return new_cols

#rolling = 13
#predictors = ["venue_code", "opp_code", "hour", "dayofweek"]
#cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]

#allmatch_rolling = allmatch_team

#newcols = rolling_average_columns(cols, rolling)
#allmatch_rolling = allmatch_team.groupby("team").apply(lambda x: rolling_averages(x, cols, newcols, rolling))
#allmatch_rolling = allmatch_rolling.droplevel('team')

#allmatch_rolling.head()

,day,gf,ga,opponent,xg,xga,poss,sh,sot,dist,...,opp_code,target,gf_rolling number_13,ga_rolling number_13,sh_rolling number_13,sot_rolling number_13,dist_rolling number_13,fk_rolling number_13,pk_rolling number_13,pkatt_rolling number_13
13,Sat,2.0,0.0,Leicester City,0.7,1.3,36.0,9.0,5.0,16.7,...,11,1,1.153846,1.307692,13.923077,4.230769,17.323077,0.615385,0.0,0.076923
14,Sun,1.0,0.0,Watford,1.6,0.5,60.0,13.0,5.0,21.1,...,22,1,1.307692,1.153846,13.769231,4.461538,17.315385,0.538462,0.0,0.076923
15,Sat,0.0,4.0,Liverpool,0.4,4.3,38.0,5.0,3.0,19.9,...,12,0,1.384615,1.076923,13.384615,4.384615,17.584615,0.615385,0.0,0.153846
16,Sat,2.0,0.0,Newcastle Utd,2.5,0.3,66.0,24.0,6.0,17.8,...,16,1,1.153846,1.307692,13.307692,4.384615,18.023077,0.615385,0.0,0.153846
17,Thu,2.0,3.0,Manchester Utd,1.4,1.9,56.0,17.0,7.0,16.5,...,15,0,1.153846,1.307692,13.923077,4.461538,17.792308,0.615385,0.0,0.153846


In [74]:
#df = pd.DataFrame({'B': [0, 1, 2, 3, 4]})
#df['Bavg2'] = df.B.shift(1).rolling(window=2).mean()
#df = df.dropna(subset='Bavg2')
#https://fbref.com/en/comps/9/Premier-League-Stats

teamdict = {
"Wolverhampton Wanderers": "Wolves",
"Brighton and Hove Albion": "Brighton",
"West Ham United": "West Ham",
"Tottenham Hotspur": "Tottenham",
"Manchester United": "Manchester Utd",
"Newcastle United": "Newcastle Utd",
"Sheffield United": "Sheffield Utd",
"Nottingham Forest": "Nott'ham Forest",
"West Bromwich Albion": "West Brom"
}

def getTeam(team):
  opponent = teamdict.get(team,None)
  if opponent:
    return opponent
  else:
    return team

bStop = False
filename = "/content/drive/MyDrive/ColabData/matches.csv"
try:
  match_df = pd.read_csv(filename)
except:
  print("Load failed")
  bStop = True
  match_df = None


if not bStop:
  # the team name an opponent list are initially not the same !?
  # translate team to the equivalent from oponent list 'Sheffield United -> Sheffield Utd.' etc.
  match_df['team'] = match_df.apply(lambda row: getTeam(row.team), axis=1)
  teamset = sorted(set(match_df['team'].tolist()))
  opponentset = sorted(set(match_df['opponent'].tolist()))
  allmatch_team = pd.DataFrame()

  if teamset != opponentset:
    print("Teams and Opponents not equal")
    bStop = True

if not bStop:
  for team in teamset:
    options = []
    options.append(team)
    match_team = match_df[match_df['team'].isin(options)].copy()
    match_team["venue_code"] = match_team["venue"].astype("category").cat.codes
    match_team['result_text'] = match_team.apply(lambda row: row.result, axis=1)
    selection = ['result','venue']
    match_team = pd.get_dummies(match_team, columns=selection)
    match_team['dateandtime'] = pd.to_datetime( match_team['date'] + ' ' + match_team['time'])
    match_team['hour'] = match_team['dateandtime'].dt.hour
    try:
      match_team['result'] = 1.0 * match_team['result_W'] - 1.0 * match_team['result_L']
    except:
      print(f"Can't find W, L or D for {team}")
      match_team['result'] = 0

    match_team.sort_values("dateandtime")
    match_team['dayofweek'] = match_team.apply(lambda row: row.dateandtime.weekday(), axis=1)
    match_team = match_team.dropna()
    matchedcols = []
    columnlist = ['dateandtime','hour','season','result','result',  'sh', 'sot', 'gf', 'dist', 'ga', 'day', 'xg', 'hour', 'fk', 'pk', 'poss', 'team', 'opponent','season', 'venue_code', 'result_text']
    for findcol in match_team.columns:
      for matchcol in columnlist:
        if findcol.startswith(matchcol):
          if findcol not in matchedcols:
            matchedcols.append(findcol)
    newmatch_team = match_team[matchedcols]
    allmatch_team = pd.concat([allmatch_team,newmatch_team], ignore_index=True)

if not bStop:
  allmatch_team = allmatch_team.sort_values(by=['team','dateandtime'], ignore_index=True)
  allmatch_team = allmatch_team.reset_index(drop=True)
  allmatch_team["team_code"] = allmatch_team["team"].astype("category").cat.codes
  allmatch_team["opp_code"] = allmatch_team["opponent"].astype("category").cat.codes
  allmatch_team["target"] = (allmatch_team["result_text"] == "W").astype("int")

  cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]

  allmatch_rolling = allmatch_team.copy()

  newcols3 = rolling_average_columns(cols, 3)
  allmatch_rolling = allmatch_rolling.groupby("team").apply(lambda x: rolling_averages(x, cols, newcols3, 3))
  allmatch_rolling = allmatch_rolling.droplevel('team')

  newcols5 = rolling_average_columns(cols, 5)
  allmatch_rolling = allmatch_rolling.groupby("team").apply(lambda x: rolling_averages(x, cols, newcols5, 5))
  allmatch_rolling = allmatch_rolling.droplevel('team')

  newcols1 = rolling_average_columns(cols, 1)
  allmatch_rolling = allmatch_rolling.groupby("team").apply(lambda x: rolling_averages(x, cols, newcols1, 1))
  allmatch_rolling = allmatch_rolling.droplevel('team')

  allmatch_rolling = allmatch_rolling.dropna()
  allmatch_rolling.index = range(allmatch_rolling.shape[0])
  allmatch_rolling.head()

Can't find W, L or D for West Brom


In [133]:
def make_predictions(data, predictors, description):
  rf = RandomForestClassifier(n_estimators=100, min_samples_split=10, random_state=42)
  train = data[data["dateandtime"] < '2023-07-01']
  test = data[data["dateandtime"] >= '2023-07-01']
  rf.fit( train[predictors], train["target"])
  preds = rf.predict(test[predictors])
  combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))
  accuracy  = accuracy_score(test["target"], preds)
  precision = precision_score(test["target"], preds)
  print( f"{description} | accuracy: {100*accuracy:.2f}% | precision: {100*precision:.2f}%" )
  return combined, precision, accuracy

In [153]:
from math import comb
allmatch_rolling = allmatch_team

highestprecision = 0
highestaccuracy = None
highestprecisioncombo = None
minrolling = 1
maxrolling = 20

for rolling in range(minrolling,maxrolling):
  allmatch_rolling = allmatch_team.copy()
  predictors = ["venue_code", "opp_code", "hour", "dayofweek"]
  cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt", "result"]

  newcols = rolling_average_columns(cols, rolling)
  allmatch_rolling = allmatch_team.groupby("team").apply(lambda x: rolling_averages(x, cols, newcols, rolling))
  allmatch_rolling = allmatch_rolling.droplevel('team')

  combo = f"{rolling} only"
  combined, precision, accuracy = make_predictions(allmatch_rolling, predictors, combo)
  if precision > highestprecision:
    highestprecision = precision
    highestaccuracy = accuracy
    highestprecisioncombo = combo

for rolling in range(minrolling,maxrolling):
  for rolling2 in range(rolling,maxrolling):
    predictors = ["venue_code", "opp_code", "hour", "dayofweek"]
    cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt", "result"]

    allmatch_rolling = allmatch_team.copy()

    newcols = rolling_average_columns(cols, rolling)
    allmatch_rolling = allmatch_rolling.groupby("team").apply(lambda x: rolling_averages(x, cols, newcols, rolling))
    allmatch_rolling = allmatch_rolling.droplevel('team')

    newcols = rolling_average_columns(cols, rolling)
    allmatch_rolling = allmatch_rolling.groupby("team").apply(lambda x: rolling_averages(x, cols, newcols, rolling2))
    allmatch_rolling = allmatch_rolling.droplevel('team')

    combo = f"{rolling} + {rolling2}"
    combined, precision, accuracy = make_predictions(allmatch_rolling, predictors, combo)
    if precision > highestprecision:
      highestprecision = precision
      highestaccuracy = accuracy
      highestprecisioncombo = combo

print("Best precision:")
print( f"{highestprecisioncombo} | accuracy: {100*highestaccuracy:.2f}% | precision: {100*highestprecision:.2f}%" )

1 only | accuracy: 59.42% | precision: 50.59%
2 only | accuracy: 59.04% | precision: 50.00%
3 only | accuracy: 60.53% | precision: 53.41%
4 only | accuracy: 58.45% | precision: 49.45%
5 only | accuracy: 59.84% | precision: 52.94%
6 only | accuracy: 59.89% | precision: 52.63%
7 only | accuracy: 59.13% | precision: 51.76%
8 only | accuracy: 60.55% | precision: 55.17%
9 only | accuracy: 59.50% | precision: 53.66%
10 only | accuracy: 59.00% | precision: 52.87%
11 only | accuracy: 59.05% | precision: 53.49%
12 only | accuracy: 59.94% | precision: 55.81%
13 only | accuracy: 59.15% | precision: 53.85%
14 only | accuracy: 58.36% | precision: 52.17%
15 only | accuracy: 57.55% | precision: 51.14%
16 only | accuracy: 57.88% | precision: 52.17%
17 only | accuracy: 57.64% | precision: 51.69%
18 only | accuracy: 58.84% | precision: 53.76%
19 only | accuracy: 59.59% | precision: 54.84%
1 + 1 | accuracy: 59.04% | precision: 50.00%
1 + 2 | accuracy: 60.53% | precision: 53.41%
1 + 3 | accuracy: 58.45% |

In [154]:
allmatch_rolling = allmatch_team.copy()
predictors = ["venue_code", "opp_code", "hour", "dayofweek"]
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt", "result"]
rolling = 1
rolling2 = 19
newcols = rolling_average_columns(cols, rolling)
allmatch_rolling = allmatch_rolling.groupby("team").apply(lambda x: rolling_averages(x, cols, newcols, rolling))
allmatch_rolling = allmatch_rolling.droplevel('team')

newcols = rolling_average_columns(cols, rolling2)
allmatch_rolling = allmatch_rolling.groupby("team").apply(lambda x: rolling_averages(x, cols, newcols, rolling2))
allmatch_rolling = allmatch_rolling.droplevel('team')
combo = f"{rolling} + {rolling2}"
combined, precision, accuracy = make_predictions(allmatch_rolling, predictors, combo)
combined = combined.merge(allmatch_rolling[["dateandtime","team","opponent","result"]], left_index=True, right_index=True)
merged = combined.merge(combined, left_on=["dateandtime", "team"], right_on=["dateandtime", "opponent"])
merged[(merged["prediction_x"] == 1) & (merged["prediction_y"] == 0)]["actual_x"].value_counts()
predictedwincorrectly = merged[(merged["prediction_x"] == 1) & (merged["prediction_y"] == 0)]["actual_x"].value_counts()[1]
totalpredictions = merged[(merged["prediction_x"] == 1) & (merged["prediction_y"] == 0)]["actual_x"].value_counts()[0] + predictedwincorrectly
prec = 100.0 * predictedwincorrectly / totalpredictions
print(f"Precision: {predictedwincorrectly} / {totalpredictions} = {prec:.2f}%")
#agreed = merged[(merged["prediction_x"] == 1) & (merged["prediction_y"] == 0)]
#ello = agreed[agreed["team_x"] =="Arsenal"]
#agreed.value_counts()
#ello[["dateandtime","result"]].head()
#allmatch_rolling["team"]
#data[data["dateandtime"] < '2023-07-01']
#5 + 7 | accuracy: 59.94% | precision: 55.81%
#1    40
#0    34
#Name: actual_x, dtype: int64#

1 + 19 | accuracy: 61.52% | precision: 58.33%
Precision: 36 / 65 = 55.38%
